In [6]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor
from sklearn.utils import all_estimators
from sklearn.base import RegressorMixin

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [7]:
cols_cible_type = ["TurnoutTimeSeconds", "TravelTimeSeconds", "PumpSecondsOnSite"]
cols_Data = [
    ["CalYear", "HourOfCall", "Postcode_district", "Month", "DayOfWeek"],
    ["CalYear", "HourOfCall", "Postcode_district", "Month", "DayOfWeek"],
    ["CalYear", "PropertyType", "StopCode"],
]
cols_cible = [
    ["TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean", "TurnoutTimeSeconds_max"],
    ["TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max"],
    ["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max"]
    # NumPumpsAttending??
]
cols_cible_filter = [
    "TurnoutTimeSeconds_mean",
    "TravelTimeSeconds_mean",
    "PumpSecondsOnSite_mean",
]

def load_df(col_cible_type):
    df = pd.read_csv(f"../data/_df_ready_{col_cible_type}.csv", sep=";", low_memory=False)
    # Contrôle
    # display(df.head(3))
    # display(df.info())
    return df

In [8]:
# Créez le cercle de corrélation
# def draw_correlation_circle(df_charges_factorielles, pca, arrow_length=0.1, label_rotation=0):
#     fig, ax = plt.subplots(figsize=(8, 8))
#     for i, var in enumerate(df_charges_factorielles.columns):
#         x = df_charges_factorielles.loc[0, var]
#         y = df_charges_factorielles.loc[1, var]
#         ax.arrow(0, 0, x, y, head_width=arrow_length, head_length=arrow_length, fc='gray', ec='gray')
#         ax.text(x*1.15, y*1.15, var, ha='center', va='center', rotation=label_rotation, fontsize=9)
#     circle = plt.Circle((0, 0), 1, facecolor='none', edgecolor='black')
#     ax.add_artist(circle)
#     ax.set_xlim(-1.1, 1.1)
#     ax.set_ylim(-1.1, 1.1)
#     ax.set_aspect('equal', adjustable='box')
#     ax.set_xlabel('Axe 1 (PC1)')
#     ax.set_ylabel('Axe 2 (PC2)')
#     ax.set_title('Cercle des Corrélations')
#     plt.grid()
#     plt.show()

ignore_regressors = [
    "ExtraTreesRegressor",
    "NuSVR",
    "QuantileRegressor",
    "SVR",
    "CCA",
    "GammaRegressor",
    "GaussianProcessRegressor",
    "IsotonicRegression",
    "MultiOutputRegressor",
    "MultiTaskElasticNet",
    "MultiTaskElasticNetCV",
    "MultiTaskLasso",
    "MultiTaskLassoCV",
    "PLSCanonical",
    "RadiusNeighborsRegressor",
    "RegressorChain",
    "StackingRegressor",
    "KernelRidge",
    "ARDRegression",
    "TweedieRegressor",
]
REGRESSORS = [
    est
    for est in all_estimators()
    if (issubclass(est[1], RegressorMixin) and (not (est[0] in ignore_regressors)))
]

all_results = pd.DataFrame()

for index, name in enumerate(cols_cible_type):
    df = load_df(name)
    # X = df.drop(["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max", "TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean",
    #               "TurnoutTimeSeconds_max", "TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max", "NumPumpsAttending"], axis=1)
    # display(X.head(2))
    # print(X.shape)

    # pca = PCA()
    # data_2D = pca.fit_transform(X)
    # display(data_2D)
    # print("La part de variance expliquée est", round(pca.explained_variance_ratio_.sum(), 5))

    # plt.figure()
    # plt.xlim(0,40)
    # plt.xlabel('Nombre de composantes')
    # plt.ylabel('Part de variance expliquée')
    # plt.axhline(y = 0.95, color ='r', linestyle = '--')
    # plt.plot(pca.explained_variance_ratio_.cumsum());
    # plt.show()

    # variance_expliquee = pca.explained_variance_ratio_
    # charges_factorielles = pca.components_
    # # Créez un DataFrame pour afficher les corrélations de chaque variable avec chaque axe
    # df_charges_factorielles = pd.DataFrame(
    #     charges_factorielles,
    #     columns=X.columns,
    #     #index=["Axe 1", "Axe 2", "Axe 3", "Axe 4", "Axe 5", "Axe 6"],
    # )

    # # Affichez le DataFrame des charges factorielles
    # display(df_charges_factorielles.head(2))

    # charges_factorielles = pca.components_
    # # Appelez la fonction pour tracer le cercle de corrélation
    # draw_correlation_circle(df_charges_factorielles, pca)

    # Auto ML
    df_limited = df[df.CalYear > 6]
    print(df_limited.shape)
    # Crée X sans les target
    cols_to_remove = [item for sublist in cols_cible for item in sublist] + [
        "NumPumpsAttending"
    ]
    print("cols_to_remove", cols_to_remove)
    X = df_limited.drop(cols_to_remove, axis=1)
    # Ne conserve que certaines colonnes explicatives
    # Crée une liste des colonnes à conserver
    cols_to_keep = [
        col
        for col in X.columns
        if any(substring in col for substring in cols_Data[index])
    ]
    print("cols_to_keep", cols_to_keep)
    # Conserver uniquement ces colonnes
    X = X[cols_to_keep]

    display(X.head(2))
    print(X.shape)

    for col_cible in cols_cible[index]:
        if not(col_cible in cols_cible_filter):
            continue
        print("cible", col_cible)
        y = df_limited[col_cible]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        myREGRESSORS = REGRESSORS
        print(myREGRESSORS)
        # myREGRESSORS = myREGRESSORS[4:6]
        reg = LazyRegressor(
            verbose=2,
            ignore_warnings=False,
            custom_metric=None,
            regressors=myREGRESSORS,
        )
        models, predictions = reg.fit(X_train, X_test, y_train, y_test)
        # Ajoute le nom de la colonne
        models["Target"] = col_cible
        # Concaténe dans les résultats
        all_results = pd.concat([all_results, models], axis=0)
        display(models)
        models.to_csv(f"../data/_autoML_regressor_{col_cible}.csv", sep=";", index=True)
        # break
    # break

all_results.to_csv(f"../data/_autoML_regressor.csv", sep=";", index=True)

# tester automl avant et après PCA et aussi gridsearch pour comparer
# save du PCA

(1016665, 42)
cols_to_remove ['TurnoutTimeSeconds_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max', 'TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeSeconds_max', 'PumpSecondsOnSite_min', 'PumpSecondsOnSite_mean', 'PumpSecondsOnSite_max', 'NumPumpsAttending']
cols_to_keep ['CalYear', 'HourOfCall_0', 'HourOfCall_1', 'HourOfCall_2', 'HourOfCall_3', 'HourOfCall_4', 'Postcode_district_0', 'Postcode_district_1', 'Postcode_district_2', 'Postcode_district_3', 'Postcode_district_4', 'Month_0', 'Month_1', 'Month_2', 'Month_3', 'DayOfWeek_0', 'DayOfWeek_1', 'DayOfWeek_2']


,CalYear,HourOfCall_0,HourOfCall_1,HourOfCall_2,HourOfCall_3,HourOfCall_4,Postcode_district_0,Postcode_district_1,Postcode_district_2,Postcode_district_3,Postcode_district_4,Month_0,Month_1,Month_2,Month_3,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2
574862,7,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1
574863,7,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1


(1016665, 18)
cible TurnoutTimeSeconds_mean
[('AdaBoostRegressor', <class 'sklearn.ensemble._weight_boosting.AdaBoostRegressor'>), ('BaggingRegressor', <class 'sklearn.ensemble._bagging.BaggingRegressor'>), ('BayesianRidge', <class 'sklearn.linear_model._bayes.BayesianRidge'>), ('DecisionTreeRegressor', <class 'sklearn.tree._classes.DecisionTreeRegressor'>), ('DummyRegressor', <class 'sklearn.dummy.DummyRegressor'>), ('ElasticNet', <class 'sklearn.linear_model._coordinate_descent.ElasticNet'>), ('ElasticNetCV', <class 'sklearn.linear_model._coordinate_descent.ElasticNetCV'>), ('ExtraTreeRegressor', <class 'sklearn.tree._classes.ExtraTreeRegressor'>), ('GradientBoostingRegressor', <class 'sklearn.ensemble._gb.GradientBoostingRegressor'>), ('HistGradientBoostingRegressor', <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingRegressor'>), ('HuberRegressor', <class 'sklearn.linear_model._huber.HuberRegressor'>), ('KNeighborsRegressor', <class 'sklearn.nei

  3%|▎         | 1/35 [00:11<06:19, 11.16s/it]

{'Model': 'AdaBoostRegressor', 'R-Squared': 0.09766274161158073, 'Adjusted R-Squared': 0.09758285497981423, 'RMSE': np.float64(35.136648918186495), 'Time taken': 11.16460108757019}


...

 94%|█████████▍| 33/35 [1:03:34<02:15, 67.82s/it]

{'Model': 'TheilSenRegressor', 'R-Squared': 0.011116612361299016, 'Adjusted R-Squared': 0.011053384212236117, 'RMSE': np.float64(1485.1513229622353), 'Time taken': 189.56537461280823}


100%|██████████| 35/35 [1:03:35<00:00, 109.01s/it]

{'Model': 'TransformedTargetRegressor', 'R-Squared': 0.14440871277056144, 'Adjusted R-Squared': 0.14435400717622937, 'RMSE': np.float64(1381.437822815433), 'Time taken': 0.856342077255249}
VotingRegressor model failed to execute
VotingRegressor.__init__() missing 1 required positional argument: 'estimators'


,Adjusted R-Squared,R-Squared,RMSE,Time Taken,Target
Model,,,,,
HistGradientBoostingRegressor,0.23,0.23,1314.48,3.33,PumpSecondsOnSite_mean
RandomForestRegressor,0.22,0.22,1317.28,67.20,PumpSecondsOnSite_mean
MLPRegressor,0.22,0.22,1317.81,656.52,PumpSecondsOnSite_mean
BaggingRegressor,0.22,0.22,1317.99,7.33,PumpSecondsOnSite_mean
DecisionTreeRegressor,0.22,0.22,1319.87,1.20,PumpSecondsOnSite_mean
ExtraTreeRegressor,0.22,0.22,1320.81,1.22,PumpSecondsOnSite_mean
GradientBoostingRegressor,0.21,0.21,1326.14,34.04,PumpSecondsOnSite_mean
AdaBoostRegressor,0.15,0.15,1375.09,7.58,PumpSecondsOnSite_mean
Lars,0.14,0.14,1381.44,0.42,PumpSecondsOnSite_mean
